In [ ]:
import dpkt


In [ ]:
# 粗粒度

In [ ]:
# 细粒度 (IPD, Packet_Size)
def extract_pcap_s(filename, packet_num):
    
    with open(filename, 'rb') as pcap_file:
        pcap_file = dpkt.pcap.Reader(pcap_file)
        packet_list = []
        for timestamp, buffer in pcap_file:
            packet_list.append({'timestamp' : timestamp, 'size' : len(buffer)})
    
    flow_list = []
    flow = []
    for i in range(len(packet_list)):
        if (i + 1) % packet_num == 0:
            flow.append(packet_list[i])
            flow_list.append(flow)
            flow = []
    if len(flow):
        flow_list.append(flow)
    
    feature_list = []
    for flow in flow_list:
        time_list = []
        size_list = []
        for i in range(len(flow)):
            time_list.append(flow[i]['timestamp'])
            size_list.append(flow[i]['size'])
        feature_list.append(time_list + size_list)

    return feature_list
